In [ ]:
import torch
import os
from torchvision.transforms import ToTensor, Resize, Compose
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np

from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
trans = Compose([
    Resize((224, 224)),
    ToTensor(),
])

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        self.xs = []
        self.ys = []
        for imgname in tqdm(os.listdir(path)):
            if imgname[5] not in '01':
                continue
            self.xs.append(trans(Image.open(path / imgname)))
            self.ys.append(torch.tensor(float(imgname[5] != '0')))
    
    def __getitem__(self, i):
        return self.xs[i], self.ys[i]

    def __len__(self):
        return len(self.xs)

In [ ]:
from pathlib import Path

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 128
IMAGE_FOLDER = Path('/content/drive/Shareddrives/AC297R/Classification/JPEGImages')
dataset = ImageDataset(IMAGE_FOLDER)

100%|██████████| 1457/1457 [03:34<00:00,  6.79it/s]


In [ ]:
train_indices = np.random.choice(len(dataset), int(len(dataset) * 0.7), replace=False)
valid_indices = [i for i in range(len(dataset)) if i not in train_indices]
dataset_train = torch.utils.data.Subset(dataset, train_indices)
dataset_valid = torch.utils.data.Subset(dataset, valid_indices)
loader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE)#, shuffle=True, num_workers=2)
loader_valid = DataLoader(dataset_valid, batch_size=BATCH_SIZE)#, shuffle=True, num_workers=2)

In [ ]:
print(len(dataset_train), len(dataset_valid))

793 341


In [ ]:
class Net(torch.nn.Module):
    def __init__(self, pretrained_net, last_shape=1000):
        super().__init__()
        self.pretrained_net = pretrained_net
        '''
        for p in self.pretrained_net.parameters():
            p.requires_grad = False
        
        for p in list(self.pretrained_net.parameters())[-6:]:
            p.requires_grad = True
        '''
        for p in list(self.pretrained_net.parameters()):
            p.requires_grad = True

        self.relu = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(last_shape, 256)
        self.fc2 = torch.nn.Linear(256, 32)
        self.fc3 = torch.nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(self.relu(self.pretrained_net(x)))
        x = self.fc2(self.relu(x))
        x = self.fc3(self.relu(x))
        return x


model = Net(resnet50(weights=ResNet50_Weights.IMAGENET1K_V2), 1000)
model.to(DEVICE)

for name, p in model.named_parameters():
    if p.requires_grad:
        print(name, p.shape)


pretrained_net.conv1.weight torch.Size([64, 3, 7, 7])
pretrained_net.bn1.weight torch.Size([64])
pretrained_net.bn1.bias torch.Size([64])
pretrained_net.layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
pretrained_net.layer1.0.bn1.weight torch.Size([64])
pretrained_net.layer1.0.bn1.bias torch.Size([64])
pretrained_net.layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
pretrained_net.layer1.0.bn2.weight torch.Size([64])
pretrained_net.layer1.0.bn2.bias torch.Size([64])
pretrained_net.layer1.0.conv3.weight torch.Size([256, 64, 1, 1])
pretrained_net.layer1.0.bn3.weight torch.Size([256])
pretrained_net.layer1.0.bn3.bias torch.Size([256])
pretrained_net.layer1.0.downsample.0.weight torch.Size([256, 64, 1, 1])
pretrained_net.layer1.0.downsample.1.weight torch.Size([256])
pretrained_net.layer1.0.downsample.1.bias torch.Size([256])
pretrained_net.layer1.1.conv1.weight torch.Size([64, 256, 1, 1])
pretrained_net.layer1.1.bn1.weight torch.Size([64])
pretrained_net.layer1.1.bn1.bias torch.Size([64])


In [ ]:
def compute_accuracy(model):
    acc = 0
    count = 0
    with torch.no_grad():
        for x, y in loader_valid:
            y_pred = model(x.to(DEVICE))
            y_pred = torch.sigmoid(y_pred)
            y_pred = (y_pred > 0.5).cpu().numpy().astype(int)
            y_true = y.cpu().numpy().astype(int)
            # print((y_true == y_pred).shape)
            acc += (y_true.flatten() == y_pred.flatten()).sum()
            count += len(y)
    return acc / count

    
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss

loss_fn = BCEWithLogitsLoss()

optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

for epoch in range(10):
    print(epoch)
    epoch_loss = 0
    for x, y in loader_train:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y.reshape(-1, 1))
        epoch_loss = loss.item() * len(y)
        loss.backward()
        optimizer.step()

    print('valid acc =', compute_accuracy(model))
    print('epoch loss =', epoch_loss / len(dataset))


0
valid acc = 0.8269794721407625
epoch loss = 0.01242908746056666
1
valid acc = 0.8269794721407625
epoch loss = 0.0068210577838635315
2
valid acc = 0.7419354838709677
epoch loss = 0.0011615901922632987
3
valid acc = 0.7888563049853372
epoch loss = 0.0003980774912043644
4
valid acc = 0.8211143695014663
epoch loss = 0.00012744436980055964
5
valid acc = 0.8035190615835777
epoch loss = 0.00021816418149772984
6
valid acc = 0.8240469208211144
epoch loss = 2.2612765166133223e-05
7
valid acc = 0.8240469208211144
epoch loss = 1.596035840272299e-05
8
valid acc = 0.8093841642228738
epoch loss = 4.278606214894005e-05
9
valid acc = 0.8357771260997068
epoch loss = 0.00017069587508640276


In [ ]:
compute_accuracy(model)

0.8357771260997068

In [ ]:
torch.save(model, '/content/drive/Shareddrives/AC297R/acne04_pretrained_resnet50_model.pth')